In [ ]:
%matplotlib widget

In [ ]:
import numpy as np
import flammkuchen as fl
import json
from lotr.default_vals import REGRESSOR_TAU_S

from lavian_et_al_2025.imaging.imaging_classes import TwoPExperiment
from lavian_et_al_2025.imaging.imaging_functions import exp_decay_kernel
from lavian_et_al_2025.data_location import master_landmarks

In [ ]:
master = master_landmarks / "2p" / "habenula"
files = list(master.glob("*_f*"))

In [ ]:
def scale_square(square_size, radius, proj_height):
    circumf = 2*np.pi*radius
    scale_factor = circumf / proj_height
    scaled_size = square_size * scale_factor
    return scaled_size

In [ ]:
num_rows = 8
n_regs = num_rows
s_size = 1 / num_rows
s_size = 0.5 / num_rows
radius = 0.3
proj_height = [2, 1.75, 1.5, 1.25, 1.25, 1.5, 1.75, 2]
        
choices = []
for x_pos in range(num_rows):
    scaled_size = scale_square(s_size, radius, proj_height[x_pos])
    curr_choice = [0, x_pos / num_rows, 1, scaled_size]
    choices.append(curr_choice)

In [ ]:
for fish in files[30:]:
    print(fish)
    plane_list = fish / "suite2p"
    planes = list(plane_list.glob("*00*"))
    
    for path in planes:
        if not (path / 'sensory_regressors_cells.h5').exists():
            metadata_file = list(path.glob("*_metadata.json"))[0]

            with open(str(metadata_file), "r") as f:
                 metadata = json.load(f)
            stim = metadata["stimulus"]["log"]

            fs = int(metadata['imaging']['microscope_config']['scanning']['framerate'])

            dt_imaging = 1 / fs
            int_fact = 200

            traces = fl.load(path / 'filtered_traces.h5', '/detr').T
            len_rec = traces.shape[1]
            t_imaging = np.arange(len_rec)/fs

            pause_duration = stim[0]['duration'] * fs
            stim_duration = stim[1]['duration'] * fs

            n_options = num_rows
            n_rep = [metadata['stimulus']['protocol']['receptive_fields']['v06_front_cols_flashes_1x8']['n_trials']][0]
            n_trials = (n_options * n_rep) 
            position_list = np.zeros((n_trials, 4))
            for_regs = np.zeros((n_options, n_trials * 2 + 1))

            regs = np.zeros((n_regs, len_rec))
            t1 = pause_duration

            for i in range(1, n_trials * 2, 2):
                curr_trial = stim[i]['clip_mask']
                position_list[(i//2) - 1, :] = curr_trial

                for j in range(n_options):
                    if curr_trial == choices[j]:
                        for_regs[j, i-1] = 1
                        regs[j, t1:(t1 + stim_duration)] = 1

                t1 = t1 + stim_duration + pause_duration 

            tau_fs = REGRESSOR_TAU_S * fs
            kernel = np.exp(-np.arange(1000) / tau_fs)
            t_imaging_int = np.arange(len_rec*int_fact)*dt_imaging/int_fact

            regs_conv = np.zeros((n_options, len_rec))

            try:
                num_traces, len_rec = np.shape(traces)
                regs_vals = np.zeros((n_options, num_traces))

                for i in range(n_options):
                    regs_conv[i] = np.convolve(regs[i], kernel)[:np.shape(traces)[1]]

                    X = traces.T
                    Y = regs_conv[i]
                    numerator = np.dot(X.T, Y) - X.shape[0] * np.nanmean(X, 0) * np.nanmean(Y)
                    denominator = (X.shape[0] - 1) * np.nanstd(X, 0) * np.nanstd(Y)
                    regs_vals[i] = numerator / denominator


                d = {'regressors': regs,
                     'regressors_conv': regs_conv,
                     'regressors_values': regs_vals,
                }
                fl.save(path / 'sensory_regressors_cells.h5', d)

                regs_vals = np.zeros((n_options, num_traces))

                for i in range(n_options):
                    regs_conv[i] = np.convolve(regs[i], kernel)[:np.shape(traces)[1]]

                    X = traces.T
                    Y = regs_conv[i]
                    numerator = np.dot(X.T, Y) - X.shape[0] * np.nanmean(X, 0) * np.nanmean(Y)
                    denominator = (X.shape[0] - 1) * np.nanstd(X, 0) * np.nanstd(Y)
                    regs_vals[i] = numerator / denominator


                d = {'regressors': regs,
                     'regressors_conv': regs_conv,
                     'regressors_values': regs_vals,
                }
                fl.save(path / 'sensory_regressors.h5', d)

            except:
                for i in range(n_options):
                    regs_conv[i] = np.convolve(regs[i], kernel)[:np.shape(traces)[1]]

                d = {'regressors': regs,
                     'regressors_conv': regs_conv,
                }

                fl.save(path / 'sensory_regressors_partial.h5', d)